In [1]:
import os
from typing import Optional, Text, List
from absl import logging
from ml_metadata.proto import metadata_store_pb2
import tfx
import tensorflow as tf
from tfx.utils.dsl_utils import csv_input, external_input
from tfx.orchestration import pipeline
from tfx.components import StatisticsGen, SchemaGen
from tfx.components.example_gen.csv_example_gen.component import CsvExampleGen
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext


logging.set_verbosity(logging.INFO)
logging.info('TensorFlow version: {}'.format(tf.__version__))
logging.info('TFX version: {}'.format(tfx.__version__))

/usr/local/lib/python3.7/site-packages/tfx/orchestration/data_types.py:191: UserWarning: RuntimeParameter is only supported on KubeflowDagRunner currently.
  warnings.warn('RuntimeParameter is only supported on KubeflowDagRunner '
INFO:absl:TensorFlow version: 2.2.0
INFO:absl:TFX version: 0.22.0


In [10]:
PIPELINE_NAME = 'my_pipeline'
PIPELINE_ROOT = os.path.join('.', 'my_pipeline_output')
METADATA_PATH = os.path.join('.', 'tfx_metadata', PIPELINE_NAME, 'metadata.db')

components = []
data_dir = "/app/data/"


In [11]:
logging.info("Creating component to read csv")
data_path = external_input(os.path.join(data_dir))
example_gen = CsvExampleGen(data_path)
logging.info("Component was created")

INFO:absl:Creating component to read csv
INFO:absl:Component was created


In [12]:
data_path


Channel(
    type_name: ExternalArtifact
    artifacts: [Artifact(type_name: ExternalArtifact, uri: /app/data/, id: 0)]
)

In [13]:
# OBJ: ver como usar o statisticsgen para ver se leu certo o csv
statistics_gen = StatisticsGen(examples=example_gen.outputs['examples'])
schema_gen = SchemaGen(statistics=statistics_gen.outputs['statistics'], infer_feature_shape=False)

In [14]:
components.append(example_gen)
components.append(statistics_gen)
components.append(schema_gen)

In [15]:
logging.info("Creating pipeline")
pipe = pipeline.Pipeline(
    pipeline_name=PIPELINE_NAME,
    pipeline_root=PIPELINE_ROOT,
    components=components,
    enable_cache=True,
)

INFO:absl:Creating pipeline


In [16]:
example_gen


CsvExampleGen(spec: <tfx.types.standard_component_specs.FileBasedExampleGenSpec object at 0x40c00e15d0>, executor_spec: <tfx.components.base.executor_spec.ExecutorClassSpec object at 0x40acbaaa10>, driver_class: <class 'tfx.components.example_gen.driver.Driver'>, component_id: CsvExampleGen, inputs: {'input': Channel(
    type_name: ExternalArtifact
    artifacts: [Artifact(type_name: ExternalArtifact, uri: /app/data/, id: 0)]
)}, outputs: {'examples': Channel(
    type_name: Examples
    artifacts: [Artifact(type_name: Examples, uri: , id: 0)]
)})

In [17]:
logging.info("Running pipeline")
context = InteractiveContext()
context.run(example_gen)

INFO:absl:Running pipeline
INFO:absl:Running driver for CsvExampleGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:select span = None


2023-09-04 18:00:33.999800: W ml_metadata/metadata_store/rdbms_metadata_access_object.cc:484] No property is defined for the Type
INFO:absl:Running executor for CsvExampleGen
INFO:absl:Generating examples.
INFO:absl:Using 1 process(es) for Beam pipeline execution.
INFO:absl:Processing input csv data /app/data/* to TFExample.
INFO:absl:Examples generated.
INFO:absl:Running publisher for CsvExampleGen
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: Channel(
            type_name: Examples
            artifacts: [Artifact(type_name: Examples, uri: /tmp/tfx-interactive-2023-09-04T18_00_33.887269-ha1fpq8g/CsvExampleGen/examples/1, id: 2)]
        ))